# **Adversarial Robustness of VGG-16, ResNet18, and EfficientNet-B0 on CIFAR-10**
# **Pretrain=True, 30 epoch**

# 

## **1. Import Required Libraries**

In [1]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is available
print(torch.cuda.get_device_name(0))  # Should print the name of your GPU

True
NVIDIA GeForce RTX 4060


In [2]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchattacks
import time
import pandas as pd
from torchvision import models

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')


Using device: cuda


## **2. Load CIFAR-10 Dataset**

In [3]:

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize for model compatibility
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


# **3. Defining Models From torchvision**

## **3.1 Define VGG-16 Model**

In [4]:

# Load pre-defined VGG-16 model from torchvision
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.model = models.vgg16(pretrained=True)
        num_ftrs = self.model.classifier[6].in_features
        self.model.classifier[6] = nn.Linear(num_ftrs, 10)  # Adjust for CIFAR-10 classes

    def forward(self, x):
        return self.model(x)


## **3.2 Define ResNet18 Model**

In [5]:

# Load pre-defined ResNet18 model from torchvision
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        self.model = models.resnet18(pretrained=True)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 10)  # Adjust for CIFAR-10 classes

    def forward(self, x):
        return self.model(x)


## **3.3 Define EfficientNet-B0 Model**

In [6]:

# Load pre-defined EfficientNet-B0 model from torchvision
class EfficientNetB0(nn.Module):
    def __init__(self):
        super(EfficientNetB0, self).__init__()
        self.model = models.efficientnet_b0(pretrained=True)
        num_ftrs = self.model.classifier[1].in_features
        self.model.classifier[1] = nn.Linear(num_ftrs, 10)  # Adjust for CIFAR-10 classes

    def forward(self, x):
        return self.model(x)



In [7]:
# Initialize models
model_vgg = VGG16().to(device)
model_resnet = ResNet18().to(device)
model_efficient = EfficientNetB0().to(device)

# Define loss function and optimizers
criterion = nn.CrossEntropyLoss()
optimizer_vgg = optim.Adam(model_vgg.parameters(), lr=0.001)
optimizer_resnet = optim.Adam(model_resnet.parameters(), lr=0.001)
optimizer_efficient = optim.Adam(model_efficient.parameters(), lr=0.001)

c:\Users\cheey\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\cheey\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\cheey\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date w

In [8]:
model_vgg=torch.load("model_vgg_30.pth", weights_only=False)
model_resnet=torch.load("model_resnet_30.pth", weights_only=False)
model_efficient=torch.load("model_efficient_30.pth", weights_only=False)
model_vgg.eval()
model_resnet.eval()
model_efficient.eval()
print("Models loaded successfully")

Models loaded successfully


## **4. Testing Transferability**

In [9]:
# Function to generate adversarial examples for Model X
def generate_adversarial_examples(model, attack):
    model.eval()
    adv_examples = []
    original_labels = []
    
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)

        adv_images = attack(images, labels).detach().cpu()
        torch.cuda.empty_cache()
        
        adv_examples.append(adv_images)
        original_labels.append(labels.cpu())

    return torch.cat(adv_examples), torch.cat(original_labels)

In [10]:
# Function to evaluate adversarial success rate on different models
def evaluate_transferability(model, adv_examples, labels, model_name):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for i in range(0, len(adv_examples), 16):  # Process batch of 16 images at a time
            batch_images = adv_examples[i:i+16].to(device)
            batch_labels = labels[i:i+16].to(device)

            outputs = model(batch_images)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == batch_labels).sum().item()
            total += batch_labels.size(0)
            
            torch.cuda.empty_cache()  # Free up memory

    transfer_success_rate = 100 * (1 - correct / total)
    print(f'Transfer Success Rate on {model_name}: {transfer_success_rate:.2f}%')
    return transfer_success_rate

# **4.1.1 Use FGSM to attack VGG-16 to create adversarial samples**

In [ ]:
# Apply FGSM attack to VGG-16
fgsm_attack = torchattacks.FGSM(model_vgg, eps=0.03)
adv_examples_vgg, labels_vgg = generate_adversarial_examples(model_vgg, fgsm_attack)

In [12]:
transfer_A_to_A = evaluate_transferability(model_vgg, adv_examples_vgg, labels_vgg, "VGG-16")
transfer_A_to_B = evaluate_transferability(model_resnet, adv_examples_vgg, labels_vgg, "ResNet18")
transfer_A_to_C = evaluate_transferability(model_efficient, adv_examples_vgg, labels_vgg, "EfficientNet-B0")

Transfer Success Rate on VGG-16: 69.16%
Transfer Success Rate on ResNet18: 46.38%
Transfer Success Rate on EfficientNet-B0: 57.40%


In [13]:
# Create transferability matrix
transfer_matrix = pd.DataFrame({
    "VGG-16 (A)": [transfer_A_to_A, transfer_A_to_B, transfer_A_to_C],
    "ResNet18 (B)": ["-", "-", "-"],  
    "EfficientNet-B0 (C)": ["-", "-", "-"]
}, index=["From VGG-16 (A)", "From ResNet18 (B)", "From EfficientNet-B0 (C)"])

print("Transferability Matrix:")
print(transfer_matrix)

Transferability Matrix:
                          VGG-16 (A) ResNet18 (B) EfficientNet-B0 (C)
From VGG-16 (A)                69.16            -                   -
From ResNet18 (B)              46.38            -                   -
From EfficientNet-B0 (C)       57.40            -                   -


# **4.1.2 Use FGSM to attack ResNet18 to create adversarial samples**

torch.cuda.empty_cache()
gc.collect()

In [ ]:
# Apply FGSM attack to ResNet18
fgsm_attack = torchattacks.FGSM(model_resnet, eps=0.03)
adv_examples_resnet, labels_resnet = generate_adversarial_examples(model_resnet, fgsm_attack)

In [15]:
transfer_B_to_A = evaluate_transferability(model_vgg, adv_examples_resnet, labels_resnet, "VGG-16")
transfer_B_to_B = evaluate_transferability(model_resnet, adv_examples_resnet, labels_resnet, "ResNet18")
transfer_B_to_C = evaluate_transferability(model_efficient, adv_examples_resnet, labels_resnet, "EfficientNet-B0")

Transfer Success Rate on VGG-16: 64.63%
Transfer Success Rate on ResNet18: 77.87%
Transfer Success Rate on EfficientNet-B0: 58.98%


In [16]:
# Create transferability matrix
transfer_matrix = pd.DataFrame({
    "VGG-16 (A)": [transfer_A_to_A, transfer_A_to_B, transfer_A_to_C],
    "ResNet18 (B)": [transfer_B_to_A, transfer_B_to_B, transfer_B_to_C],  
    "EfficientNet-B0 (C)": ["-", "-", "-"]
}, index=["From VGG-16 (A)", "From ResNet18 (B)", "From EfficientNet-B0 (C)"])

print("Transferability Matrix:")
print(transfer_matrix)

Transferability Matrix:
                          VGG-16 (A)  ResNet18 (B) EfficientNet-B0 (C)
From VGG-16 (A)                69.16         64.63                   -
From ResNet18 (B)              46.38         77.87                   -
From EfficientNet-B0 (C)       57.40         58.98                   -


# **4.1.3 Use FGSM to attack EfficientNet-B0 to create adversarial samples**

In [ ]:
# Apply FGSM attack to EfficientNet-B0
fgsm_attack = torchattacks.FGSM(model_efficient, eps=0.03)
adv_examples_efficient, labels_efficient = generate_adversarial_examples(model_efficient, fgsm_attack)

In [18]:
transfer_C_to_A = evaluate_transferability(model_vgg, adv_examples_efficient, labels_efficient, "VGG-16")
transfer_C_to_B = evaluate_transferability(model_resnet, adv_examples_efficient, labels_efficient, "ResNet18")
transfer_C_to_C = evaluate_transferability(model_efficient, adv_examples_efficient, labels_efficient, "EfficientNet-B0")

Transfer Success Rate on VGG-16: 55.38%
Transfer Success Rate on ResNet18: 45.81%
Transfer Success Rate on EfficientNet-B0: 80.13%


In [19]:
# Create transferability matrix
transfer_matrix = pd.DataFrame({
    "VGG-16 (A)": [transfer_A_to_A, transfer_A_to_B, transfer_A_to_C],
    "ResNet18 (B)": [transfer_B_to_A, transfer_B_to_B, transfer_B_to_C],  
    "EfficientNet-B0 (C)": [transfer_C_to_A, transfer_C_to_B, transfer_C_to_C]
}, index=["From VGG-16 (A)", "From ResNet18 (B)", "From EfficientNet-B0 (C)"])

print("Transferability Matrix:")
print(transfer_matrix)

Transferability Matrix:
                          VGG-16 (A)  ResNet18 (B)  EfficientNet-B0 (C)
From VGG-16 (A)                69.16         64.63                55.38
From ResNet18 (B)              46.38         77.87                45.81
From EfficientNet-B0 (C)       57.40         58.98                80.13


# **4.2.1 Use PGD to attack VGG16 to create adversarial samples**

In [11]:
import gc
torch.cuda.empty_cache()
gc.collect()

18

In [12]:
# Apply PGD attack to VGG-16
pgd_attack = torchattacks.PGD(model_vgg, eps=0.03, alpha=0.01, steps=40)
adv_examples_vgg, labels_vgg = generate_adversarial_examples(model_vgg, pgd_attack)

In [13]:
transfer_A_to_A = evaluate_transferability(model_vgg, adv_examples_vgg, labels_vgg, "VGG-16")
transfer_A_to_B = evaluate_transferability(model_resnet, adv_examples_vgg, labels_vgg, "ResNet18")
transfer_A_to_C = evaluate_transferability(model_efficient, adv_examples_vgg, labels_vgg, "EfficientNet-B0")

Transfer Success Rate on VGG-16: 99.91%
Transfer Success Rate on ResNet18: 46.20%
Transfer Success Rate on EfficientNet-B0: 53.16%


In [14]:
# Create transferability matrix
transfer_matrix = pd.DataFrame({
    "VGG-16 (A)": [transfer_A_to_A, transfer_A_to_B, transfer_A_to_C],
    "ResNet18 (B)": ["-", "-", "-"],  
    "EfficientNet-B0 (C)": ["-", "-", "-"]
}, index=["From VGG-16 (A)", "From ResNet18 (B)", "From EfficientNet-B0 (C)"])

print("Transferability Matrix:")
print(transfer_matrix)

Transferability Matrix:
                          VGG-16 (A) ResNet18 (B) EfficientNet-B0 (C)
From VGG-16 (A)                99.91            -                   -
From ResNet18 (B)              46.20            -                   -
From EfficientNet-B0 (C)       53.16            -                   -


# **4.2.2 Use PGD to attack ResNet18 to create adversarial samples**

In [14]:
import gc
torch.cuda.empty_cache()
gc.collect()

202

In [ ]:
# Apply PGD attack to ResNet18
pgd_attack = torchattacks.PGD(model_resnet, eps=0.03, alpha=0.01, steps=40)
adv_examples_resnet, labels_resnet = generate_adversarial_examples(model_resnet, pgd_attack)

In [17]:
transfer_B_to_A = evaluate_transferability(model_vgg, adv_examples_resnet, labels_resnet, "VGG-16")
transfer_B_to_B = evaluate_transferability(model_resnet, adv_examples_resnet, labels_resnet, "ResNet18")
transfer_B_to_C = evaluate_transferability(model_efficient, adv_examples_resnet, labels_resnet, "EfficientNet-B0")

Transfer Success Rate on VGG-16: 66.86%
Transfer Success Rate on ResNet18: 99.92%
Transfer Success Rate on EfficientNet-B0: 65.22%


In [18]:
# Create transferability matrix
transfer_matrix = pd.DataFrame({
    "VGG-16 (A)": [transfer_A_to_A, transfer_A_to_B, transfer_A_to_C],
    "ResNet18 (B)": [transfer_B_to_A, transfer_B_to_B, transfer_B_to_C],  
    "EfficientNet-B0 (C)": ["-", "-", "-"]
}, index=["From VGG-16 (A)", "From ResNet18 (B)", "From EfficientNet-B0 (C)"])

print("Transferability Matrix:")
print(transfer_matrix)

Transferability Matrix:
                          VGG-16 (A)  ResNet18 (B) EfficientNet-B0 (C)
From VGG-16 (A)                99.90         66.86                   -
From ResNet18 (B)              46.41         99.92                   -
From EfficientNet-B0 (C)       52.80         65.22                   -


# **4.2.3 Use PGD to attack EfficientNet-B0 to create adversarial samples**

In [19]:

torch.cuda.empty_cache()
gc.collect()

640

In [20]:
# Apply PGD attack to EfficientNet-B0
pgd_attack = torchattacks.PGD(model_efficient, eps=0.03, alpha=0.01, steps=40)
adv_examples_efficient, labels_efficient = generate_adversarial_examples(model_efficient, pgd_attack)

In [21]:
transfer_C_to_A = evaluate_transferability(model_vgg, adv_examples_efficient, labels_efficient, "VGG-16")
transfer_C_to_B = evaluate_transferability(model_resnet, adv_examples_efficient, labels_efficient, "ResNet18")
transfer_C_to_C = evaluate_transferability(model_efficient, adv_examples_efficient, labels_efficient, "EfficientNet-B0")

Transfer Success Rate on VGG-16: 54.44%
Transfer Success Rate on ResNet18: 44.98%
Transfer Success Rate on EfficientNet-B0: 100.00%


In [22]:
# Create transferability matrix
transfer_matrix = pd.DataFrame({
    "VGG-16 (A)": [transfer_A_to_A, transfer_A_to_B, transfer_A_to_C],
    "ResNet18 (B)": [transfer_B_to_A, transfer_B_to_B, transfer_B_to_C],  
    "EfficientNet-B0 (C)": [transfer_C_to_A, transfer_C_to_B, transfer_C_to_C]
}, index=["From VGG-16 (A)", "From ResNet18 (B)", "From EfficientNet-B0 (C)"])

print("Transferability Matrix:")
print(transfer_matrix)

Transferability Matrix:
                          VGG-16 (A)  ResNet18 (B)  EfficientNet-B0 (C)
From VGG-16 (A)                99.90         66.86                54.44
From ResNet18 (B)              46.41         99.92                44.98
From EfficientNet-B0 (C)       52.80         65.22               100.00


# **4.3.1 Use DeepFool to attack VGG16 to create adversarial samples**

In [12]:
import gc
torch.cuda.empty_cache()
gc.collect()

18

In [13]:
# Apply DeepFool attack to VGG-16
deepfool_attack = torchattacks.DeepFool(model_vgg, steps=50)
adv_examples_vgg, labels_vgg = generate_adversarial_examples(model_vgg, deepfool_attack)

In [14]:
transfer_A_to_A = evaluate_transferability(model_vgg, adv_examples_vgg, labels_vgg, "VGG-16")
transfer_A_to_B = evaluate_transferability(model_resnet, adv_examples_vgg, labels_vgg, "ResNet18")
transfer_A_to_C = evaluate_transferability(model_efficient, adv_examples_vgg, labels_vgg, "EfficientNet-B0")

Transfer Success Rate on VGG-16: 98.94%
Transfer Success Rate on ResNet18: 35.29%
Transfer Success Rate on EfficientNet-B0: 33.02%


In [15]:
# Create transferability matrix
transfer_matrix = pd.DataFrame({
    "VGG-16 (A)": [transfer_A_to_A, transfer_A_to_B, transfer_A_to_C],
    "ResNet18 (B)": ["-", "-", "-"],  
    "EfficientNet-B0 (C)": ["-", "-", "-"]
}, index=["From VGG-16 (A)", "From ResNet18 (B)", "From EfficientNet-B0 (C)"])

print("Transferability Matrix:")
print(transfer_matrix)

Transferability Matrix:
                          VGG-16 (A) ResNet18 (B) EfficientNet-B0 (C)
From VGG-16 (A)                98.94            -                   -
From ResNet18 (B)              35.29            -                   -
From EfficientNet-B0 (C)       33.02            -                   -


# **4.3.2 Use DeepFool to attack ResNet18 to create adversarial samples**

In [17]:

torch.cuda.empty_cache()
gc.collect()

0

In [18]:
# Apply DeepFool attack to ResNet18
deepfool_attack = torchattacks.DeepFool(model_resnet, steps=50)
adv_examples_resnet, labels_resnet = generate_adversarial_examples(model_resnet, deepfool_attack)

In [19]:
transfer_B_to_A = evaluate_transferability(model_vgg, adv_examples_resnet, labels_resnet, "VGG-16")
transfer_B_to_B = evaluate_transferability(model_resnet, adv_examples_resnet, labels_resnet, "ResNet18")
transfer_B_to_C = evaluate_transferability(model_efficient, adv_examples_resnet, labels_resnet, "EfficientNet-B0")

Transfer Success Rate on VGG-16: 54.68%
Transfer Success Rate on ResNet18: 83.65%
Transfer Success Rate on EfficientNet-B0: 42.85%


In [20]:
# Create transferability matrix
transfer_matrix = pd.DataFrame({
    "VGG-16 (A)": [transfer_A_to_A, transfer_A_to_B, transfer_A_to_C],
    "ResNet18 (B)": [transfer_B_to_A, transfer_B_to_B, transfer_B_to_C],  
    "EfficientNet-B0 (C)": ["-", "-", "-"]
}, index=["From VGG-16 (A)", "From ResNet18 (B)", "From EfficientNet-B0 (C)"])

print("Transferability Matrix:")
print(transfer_matrix)

Transferability Matrix:
                          VGG-16 (A)  ResNet18 (B) EfficientNet-B0 (C)
From VGG-16 (A)                98.94         54.68                   -
From ResNet18 (B)              35.29         83.65                   -
From EfficientNet-B0 (C)       33.02         42.85                   -


# **4.3.3 Use DeepFool to attack EfficientNet-B0 to create adversarial samples**

In [21]:

torch.cuda.empty_cache()
gc.collect()

760

In [22]:
# Apply DeepFool attack to EfficientNet-B0
deepfool_attack = torchattacks.DeepFool(model_efficient, steps=50)
adv_examples_efficient, labels_efficient = generate_adversarial_examples(model_efficient, deepfool_attack)

In [23]:
transfer_C_to_A = evaluate_transferability(model_vgg, adv_examples_efficient, labels_efficient, "VGG-16")
transfer_C_to_B = evaluate_transferability(model_resnet, adv_examples_efficient, labels_efficient, "ResNet18")
transfer_C_to_C = evaluate_transferability(model_efficient, adv_examples_efficient, labels_efficient, "EfficientNet-B0")

Transfer Success Rate on VGG-16: 53.08%
Transfer Success Rate on ResNet18: 42.67%
Transfer Success Rate on EfficientNet-B0: 78.06%


In [24]:
# Create transferability matrix
transfer_matrix = pd.DataFrame({
    "VGG-16 (A)": [transfer_A_to_A, transfer_A_to_B, transfer_A_to_C],
    "ResNet18 (B)": [transfer_B_to_A, transfer_B_to_B, transfer_B_to_C],  
    "EfficientNet-B0 (C)": [transfer_C_to_A, transfer_C_to_B, transfer_C_to_C]
}, index=["From VGG-16 (A)", "From ResNet18 (B)", "From EfficientNet-B0 (C)"])

print("Transferability Matrix:")
print(transfer_matrix)

Transferability Matrix:
                          VGG-16 (A)  ResNet18 (B)  EfficientNet-B0 (C)
From VGG-16 (A)                98.94         54.68                53.08
From ResNet18 (B)              35.29         83.65                42.67
From EfficientNet-B0 (C)       33.02         42.85                78.06


# **4.4.1 Use BIM to attack VGG16 to create adversarial samples**

In [26]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [27]:
# Apply BIM attack to VGG-16
bim_attack = torchattacks.BIM(model_vgg, eps=0.03, alpha=0.01, steps=10)
adv_examples_vgg, labels_vgg = generate_adversarial_examples(model_vgg, bim_attack)

In [28]:
transfer_A_to_A = evaluate_transferability(model_vgg, adv_examples_vgg, labels_vgg, "VGG-16")
transfer_A_to_B = evaluate_transferability(model_resnet, adv_examples_vgg, labels_vgg, "ResNet18")
transfer_A_to_C = evaluate_transferability(model_efficient, adv_examples_vgg, labels_vgg, "EfficientNet-B0")

Transfer Success Rate on VGG-16: 97.14%
Transfer Success Rate on ResNet18: 10.08%
Transfer Success Rate on EfficientNet-B0: 9.50%


In [29]:
# Create transferability matrix
transfer_matrix = pd.DataFrame({
    "VGG-16 (A)": [transfer_A_to_A, transfer_A_to_B, transfer_A_to_C],
    "ResNet18 (B)": ["-", "-", "-"],  
    "EfficientNet-B0 (C)": ["-", "-", "-"]
}, index=["From VGG-16 (A)", "From ResNet18 (B)", "From EfficientNet-B0 (C)"])

print("Transferability Matrix:")
print(transfer_matrix)

Transferability Matrix:
                          VGG-16 (A) ResNet18 (B) EfficientNet-B0 (C)
From VGG-16 (A)                97.14            -                   -
From ResNet18 (B)              10.08            -                   -
From EfficientNet-B0 (C)        9.50            -                   -


# **4.4.2 Use BIM to attack ResNet18 to create adversarial samples**

In [31]:

torch.cuda.empty_cache()
gc.collect()

0

In [32]:
# Apply BIM attack to ResNet18
bim_attack = torchattacks.BIM(model_resnet, eps=0.03, alpha=0.01, steps=10)
adv_examples_vgg, labels_vgg = generate_adversarial_examples(model_resnet, bim_attack)

In [33]:
transfer_B_to_A = evaluate_transferability(model_vgg, adv_examples_resnet, labels_resnet, "VGG-16")
transfer_B_to_B = evaluate_transferability(model_resnet, adv_examples_resnet, labels_resnet, "ResNet18")
transfer_B_to_C = evaluate_transferability(model_efficient, adv_examples_resnet, labels_resnet, "EfficientNet-B0")

Transfer Success Rate on VGG-16: 54.68%
Transfer Success Rate on ResNet18: 83.65%
Transfer Success Rate on EfficientNet-B0: 42.85%


In [34]:
# Create transferability matrix
transfer_matrix = pd.DataFrame({
    "VGG-16 (A)": [transfer_A_to_A, transfer_A_to_B, transfer_A_to_C],
    "ResNet18 (B)": [transfer_B_to_A, transfer_B_to_B, transfer_B_to_C],  
    "EfficientNet-B0 (C)": ["-", "-", "-"]
}, index=["From VGG-16 (A)", "From ResNet18 (B)", "From EfficientNet-B0 (C)"])

print("Transferability Matrix:")
print(transfer_matrix)

Transferability Matrix:
                          VGG-16 (A)  ResNet18 (B) EfficientNet-B0 (C)
From VGG-16 (A)                97.14         54.68                   -
From ResNet18 (B)              10.08         83.65                   -
From EfficientNet-B0 (C)        9.50         42.85                   -


# **4.4.3 Use BIM to attack EfficientNet18 to create adversarial samples**

In [36]:

torch.cuda.empty_cache()
gc.collect()

0

In [37]:
# Apply BIM attack to EfficientNet-B0
bim_attack = torchattacks.BIM(model_efficient, eps=0.03, alpha=0.01, steps=10)
adv_examples_vgg, labels_vgg = generate_adversarial_examples(model_efficient, bim_attack)

In [38]:
transfer_C_to_A = evaluate_transferability(model_vgg, adv_examples_efficient, labels_efficient, "VGG-16")
transfer_C_to_B = evaluate_transferability(model_resnet, adv_examples_efficient, labels_efficient, "ResNet18")
transfer_C_to_C = evaluate_transferability(model_efficient, adv_examples_efficient, labels_efficient, "EfficientNet-B0")

Transfer Success Rate on VGG-16: 53.08%
Transfer Success Rate on ResNet18: 42.67%
Transfer Success Rate on EfficientNet-B0: 78.06%


In [39]:
# Create transferability matrix
transfer_matrix = pd.DataFrame({
    "VGG-16 (A)": [transfer_A_to_A, transfer_A_to_B, transfer_A_to_C],
    "ResNet18 (B)": [transfer_B_to_A, transfer_B_to_B, transfer_B_to_C],  
    "EfficientNet-B0 (C)": [transfer_C_to_A, transfer_C_to_B, transfer_C_to_C]
}, index=["From VGG-16 (A)", "From ResNet18 (B)", "From EfficientNet-B0 (C)"])

print("Transferability Matrix:")
print(transfer_matrix)

Transferability Matrix:
                          VGG-16 (A)  ResNet18 (B)  EfficientNet-B0 (C)
From VGG-16 (A)                97.14         54.68                53.08
From ResNet18 (B)              10.08         83.65                42.67
From EfficientNet-B0 (C)        9.50         42.85                78.06


# **4.5.1 Use C&W to attack VGG16 to create adversarial samples**

In [41]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [42]:
# Apply C&W attack to VGG-16
cw_attack = torchattacks.CW(model_vgg, c=1, kappa=0, steps=100, lr=0.01)
adv_examples_vgg, labels_vgg = generate_adversarial_examples(model_vgg, cw_attack)

In [43]:
transfer_A_to_A = evaluate_transferability(model_vgg, adv_examples_vgg, labels_vgg, "VGG-16")
transfer_A_to_B = evaluate_transferability(model_resnet, adv_examples_vgg, labels_vgg, "ResNet18")
transfer_A_to_C = evaluate_transferability(model_efficient, adv_examples_vgg, labels_vgg, "EfficientNet-B0")

Transfer Success Rate on VGG-16: 99.24%
Transfer Success Rate on ResNet18: 43.61%
Transfer Success Rate on EfficientNet-B0: 42.48%


In [44]:
# Create transferability matrix
transfer_matrix = pd.DataFrame({
    "VGG-16 (A)": [transfer_A_to_A, transfer_A_to_B, transfer_A_to_C],
    "ResNet18 (B)": ["-", "-", "-"],  
    "EfficientNet-B0 (C)": ["-", "-", "-"]
}, index=["From VGG-16 (A)", "From ResNet18 (B)", "From EfficientNet-B0 (C)"])

print("Transferability Matrix:")
print(transfer_matrix)

Transferability Matrix:
                          VGG-16 (A) ResNet18 (B) EfficientNet-B0 (C)
From VGG-16 (A)                99.24            -                   -
From ResNet18 (B)              43.61            -                   -
From EfficientNet-B0 (C)       42.48            -                   -


# **4.5.2 Use C&W to attack ResNet18 to create adversarial samples**

In [46]:

torch.cuda.empty_cache()
gc.collect()

0

In [47]:
# Apply C&W attack to ResNet18
cw_attack = torchattacks.CW(model_resnet, c=1, kappa=0, steps=100, lr=0.01)
adv_examples_vgg, labels_vgg = generate_adversarial_examples(model_resnet, cw_attack)

In [48]:
transfer_B_to_A = evaluate_transferability(model_vgg, adv_examples_resnet, labels_resnet, "VGG-16")
transfer_B_to_B = evaluate_transferability(model_resnet, adv_examples_resnet, labels_resnet, "ResNet18")
transfer_B_to_C = evaluate_transferability(model_efficient, adv_examples_resnet, labels_resnet, "EfficientNet-B0")

Transfer Success Rate on VGG-16: 54.68%
Transfer Success Rate on ResNet18: 83.65%
Transfer Success Rate on EfficientNet-B0: 42.85%


In [49]:
# Create transferability matrix
transfer_matrix = pd.DataFrame({
    "VGG-16 (A)": [transfer_A_to_A, transfer_A_to_B, transfer_A_to_C],
    "ResNet18 (B)": [transfer_B_to_A, transfer_B_to_B, transfer_B_to_C],  
    "EfficientNet-B0 (C)": ["-", "-", "-"]
}, index=["From VGG-16 (A)", "From ResNet18 (B)", "From EfficientNet-B0 (C)"])

print("Transferability Matrix:")
print(transfer_matrix)

Transferability Matrix:
                          VGG-16 (A)  ResNet18 (B) EfficientNet-B0 (C)
From VGG-16 (A)                99.24         54.68                   -
From ResNet18 (B)              43.61         83.65                   -
From EfficientNet-B0 (C)       42.48         42.85                   -


# **4.5.3 Use C&W to attack EfficientNet18 to create adversarial samples**

In [56]:

torch.cuda.empty_cache()
gc.collect()

0

In [52]:
# Apply C&W attack to EfficientNet-B0
cw_attack = torchattacks.CW(model_efficient, c=1, kappa=0, steps=100, lr=0.01)
adv_examples_vgg, labels_vgg = generate_adversarial_examples(model_efficient, cw_attack)

In [53]:
transfer_C_to_A = evaluate_transferability(model_vgg, adv_examples_efficient, labels_efficient, "VGG-16")
transfer_C_to_B = evaluate_transferability(model_resnet, adv_examples_efficient, labels_efficient, "ResNet18")
transfer_C_to_C = evaluate_transferability(model_efficient, adv_examples_efficient, labels_efficient, "EfficientNet-B0")

Transfer Success Rate on VGG-16: 53.08%
Transfer Success Rate on ResNet18: 42.67%
Transfer Success Rate on EfficientNet-B0: 78.06%


In [54]:
# Create transferability matrix
transfer_matrix = pd.DataFrame({
    "VGG-16 (A)": [transfer_A_to_A, transfer_A_to_B, transfer_A_to_C],
    "ResNet18 (B)": [transfer_B_to_A, transfer_B_to_B, transfer_B_to_C],  
    "EfficientNet-B0 (C)": [transfer_C_to_A, transfer_C_to_B, transfer_C_to_C]
}, index=["From VGG-16 (A)", "From ResNet18 (B)", "From EfficientNet-B0 (C)"])

print("Transferability Matrix:")
print(transfer_matrix)

Transferability Matrix:
                          VGG-16 (A)  ResNet18 (B)  EfficientNet-B0 (C)
From VGG-16 (A)                99.24         54.68                53.08
From ResNet18 (B)              43.61         83.65                42.67
From EfficientNet-B0 (C)       42.48         42.85                78.06
